In [38]:
import numpy as np
import tomopy
import algotom.io.loadersaver as losa
import algotom.prep.correction as corr
import algotom.prep.calculation as calc
import algotom.rec.reconstruction as rec
import algotom.prep.removal as remo
import algotom.prep.filtering as filt
import algotom.util.utility as util

In [59]:
# Datei laden und Speicherort definieren
with h5py.File('phase_recos.h5', 'r') as f:
    projections = f['phase_recos'][:]
    
output_base = "H:/paul/tomo_reco_glass_beads/"

In [ ]:
# Optionale Parameter aus Beispiel übernommen
start_slice = 10
stop_slice = -1
chunk = 100  # Number of slices to be reconstructed in one go. Adjust to suit RAM or GPU memory.
ncore = None  # Number of cpu-core for parallel processing. Set to None for autoselecting.
output_format = "h5"  # "tif" or "hdf".
preprocessing = True  # Clean data before reconstruction.

In [31]:
# Rekonstruktionsparameter aus tomoreko
rotation_center = 128
rotation_axis = 0
rotation_angles_deg = np.linspace(0, 180, projections.shape[0])
rotation_angles_rad = np.deg2rad(rotation_angles_deg)
angles = np.deg2rad(rotation_angles_deg)

In [32]:
#Algo und Center berechnet
recon_method = rec.gridrec_reconstruction # Fast cpu-method. Must install Tomopy.
center = tomopy.find_center(projections, rotation_angles_rad, init=290, ind=0, tol=0.5)

In [33]:
#Werte definiert
(depth, height, width) = projections.shape

In [34]:
#Werte als beispiel
start_slice = 100
stop_slice = height - 100
step_slice = 100

In [35]:
#Parameter+Rekonstruktion Setup
if (stop_slice == -1) or (stop_slice > height):
    stop_slice = height
total_slice = stop_slice - start_slice
if output_format == ".h5":
    # float32 macht später einen Fehler: Lossy conversion from float32 to uint8, Konvertierung folgt
    recon_h5 = losa.open_hdf_stream(output_base + "/recon_data.h5",
                                     (total_slice, width, width),
                                     key_path='entry/data',
                                     data_type='float32', overwrite=True)
chunk = np.clip(chunk, 1, total_slice)
last_chunk = total_slice - chunk * (total_slice // chunk)

In [ ]:
from PIL import Image

# Nach Bsp aufgebaut
if output_format == "hdf":
    recon_hdf[start_sino - start_slice:stop_sino - start_slice] = np.moveaxis(recon_imgs, 1, 0)
else:
    for j in range(start_sino, stop_sino):
        # Konvertierung
        reconstructed_image = Image.fromarray(recon_imgs[:, j - start_sino, :])

        # Speichern
        out_file = output_base + "/rec_" + ("0000" + str(j))[-5:] + ".tif"
        reconstructed_image.save(out_file)

        # Anzeigen
        reconstructed_image.show()


In [ ]:
# Rekonstruktion aber in anderem Format
for i in np.arange(start_slice, start_slice + total_slice - last_chunk, chunk):
    start_sino = i
    stop_sino = start_sino + chunk

    # Data ohne Korrektur
    sinograms = projections[:, start_sino:stop_sino, :]

    # Rekonstruktion
    recon_imgs = recon_method(sinograms, center, angles=angles, ncore=ncore, apply_log=False)

    # Konvertieren
    recon_imgs_uint8 = (recon_imgs * 255).astype(np.uint8)

    # Anhängen an Liste
    recon_images.extend(recon_imgs_uint8)
